In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import sys
import os
from loguru import logger
from dotenv import load_dotenv
from pydantic import BaseModel
from tqdm.notebook import tqdm

import numpy as np # required for the scikit-learn pipeline to work
import pandas as pd
import plotly.express as px
import mlflow

load_dotenv()

sys.path.insert(0, '..')

from src.viz import blueq_colors

# Controller

In [3]:
class Args(BaseModel):
    testing: bool = False
    log_to_mlflow: bool = True
    experiment_name: str = "FSDS RecSys - L5 - Reco Algo"
    run_name: str = '010-baseline-popular'
    notebook_persist_dp: str = None
    random_seed: int = 41

    user_col: str = 'user_id'
    item_col: str = 'parent_asin'
    rating_col: str = 'rating'
    
    top_K: int = 100
    top_k: int = 10

    batch_size: int = 128

    def init(self):
        self.notebook_persist_dp = os.path.abspath(f"data/{self.run_name}")
        
        if not os.environ.get("MLFLOW_TRACKING_URI"):
            logger.warning(
                f"Environment variable MLFLOW_TRACKING_URI is not set. Setting self.log_to_mlflow to false."
            )
            self.log_to_mlflow = False

        if self.log_to_mlflow:
            logger.info(
                f"Setting up MLflow experiment {self.experiment_name} - run {self.run_name}..."
            )
            import mlflow

            mlflow.set_experiment(self.experiment_name)
            mlflow.start_run(run_name=self.run_name)

        return self
    
args = Args().init()

print(args.model_dump_json(indent=2))

2024-09-15 11:48:01.770 | INFO     | __main__:init:28 - Setting up MLflow experiment FSDS RecSys - L5 - Reco Algo - run 010-baseline-popular...


{
  "testing": false,
  "log_to_mlflow": true,
  "experiment_name": "FSDS RecSys - L5 - Reco Algo",
  "run_name": "010-baseline-popular",
  "notebook_persist_dp": "/home/dvquys/frostmourne/reco-algo/notebooks/data/010-baseline-popular",
  "random_seed": 41,
  "user_col": "user_id",
  "item_col": "parent_asin",
  "rating_col": "rating",
  "top_K": 100,
  "top_k": 10,
  "batch_size": 128
}


# Prep data

In [4]:
train_df = pd.read_parquet("../data/train.parquet")
val_df = pd.read_parquet("../data/val.parquet")

In [5]:
user_col = args.user_col
item_col = args.item_col
rating_col = args.rating_col

# Implememt

In [6]:
popular_items_df = (
    train_df
    .groupby(item_col, as_index=False)
    .size()
    .assign(
        rec_ranking=lambda df: df['size'].rank(method='first', ascending=False).astype(int)
    )
    .sort_values(['rec_ranking'], ascending=[True])
    .head(args.top_K)
)

popular_items_df

,parent_asin,size,rec_ranking
1737,B01N3ASPNV,527,1
2120,B077GG9D5D,317,2
3691,B087NNPYP3,245,3
822,B004RMK5QG,232,4
3371,B07YBXFDYN,229,5
...,...,...,...
1149,B00J5C3Z10,66,96
3784,B089QYP649,66,97
1710,B01MG8P418,65,98
5007,B09Y97GZG5,65,99


In [7]:
recommendations_df = (
    val_df[[user_col]].drop_duplicates().assign(tmp=1)
    .pipe(lambda df: pd.merge(
        df,
        popular_items_df.assign(tmp=1),
        on='tmp',
        how='left'
    ))
    .rename(columns={'size': 'score'})
    [[user_col, item_col, 'score', 'rec_ranking']]
)
    
recommendations_df

,user_id,parent_asin,score,rec_ranking
0,AGCL7QDBZ24RZHTSPHSQ4ZXSG3RQ,B01N3ASPNV,527,1
1,AGCL7QDBZ24RZHTSPHSQ4ZXSG3RQ,B077GG9D5D,317,2
2,AGCL7QDBZ24RZHTSPHSQ4ZXSG3RQ,B087NNPYP3,245,3
3,AGCL7QDBZ24RZHTSPHSQ4ZXSG3RQ,B004RMK5QG,232,4
4,AGCL7QDBZ24RZHTSPHSQ4ZXSG3RQ,B07YBXFDYN,229,5
...,...,...,...,...
938795,AEUDRWR7PNA6JPXZ5RTN6KCXKOBA,B00J5C3Z10,66,96
938796,AEUDRWR7PNA6JPXZ5RTN6KCXKOBA,B089QYP649,66,97
938797,AEUDRWR7PNA6JPXZ5RTN6KCXKOBA,B01MG8P418,65,98
938798,AEUDRWR7PNA6JPXZ5RTN6KCXKOBA,B09Y97GZG5,65,99


# Evaluate

## Ranking metrics

In [8]:
from src.eval import create_label_df, merge_recs_with_target

In [9]:
label_df = create_label_df(val_df)
label_df

,user_id,parent_asin,rating,rating_rank
11013,AE2WIEN3VYPFDPZ3IATSPFQQV2FQ,B094YHB1QK,1.0,1.0
2360,AH4JY7JDGBM5KWZNNS564ZZ7KCEA,B07D53QSMK,5.0,1.0
7596,AH5XFYDNDNYOSEYC7VZ6ORVNS3UA,B07XHMFCJ2,5.0,1.0
7647,AG3N3EMFIFGW66WOIC4MW55IUS6Q,B0BXQH38S6,4.0,1.0
21493,AGJAX3U73M52QLRSR4VYWZXEENMA,B07ZJ6RY1W,5.0,1.0
...,...,...,...,...
501,AETPWUNCW3KZDTKNUQGS5YYKNUEA,B08JTNL426,3.0,35.0
537,AETPWUNCW3KZDTKNUQGS5YYKNUEA,B008CP6MA2,2.0,36.0
531,AETPWUNCW3KZDTKNUQGS5YYKNUEA,B07BZP7HML,2.0,37.0
517,AETPWUNCW3KZDTKNUQGS5YYKNUEA,B008FPMBNG,2.0,38.0


In [10]:
eval_df = merge_recs_with_target(recommendations_df, label_df, k=args.top_K)
eval_df

,user_id,parent_asin,score,rec_ranking,rating,rating_rank
48,AE22LPCN47WUTHSG67R6SKN4A4MQ,B01N3ASPNV,527.0,1,0,NaN
60,AE22LPCN47WUTHSG67R6SKN4A4MQ,B077GG9D5D,317.0,2,0,NaN
80,AE22LPCN47WUTHSG67R6SKN4A4MQ,B087NNPYP3,245.0,3,0,NaN
2,AE22LPCN47WUTHSG67R6SKN4A4MQ,B004RMK5QG,232.0,4,0,NaN
74,AE22LPCN47WUTHSG67R6SKN4A4MQ,B07YBXFDYN,229.0,5,0,NaN
...,...,...,...,...,...,...
952924,AHZZZY2XVWEUJUTYPGGL4WXH6CSA,B089QYP649,66.0,97,0,NaN
952887,AHZZZY2XVWEUJUTYPGGL4WXH6CSA,B01MG8P418,65.0,98,0,NaN
952934,AHZZZY2XVWEUJUTYPGGL4WXH6CSA,B09Y97GZG5,65.0,99,0,NaN
952866,AHZZZY2XVWEUJUTYPGGL4WXH6CSA,B00KSQHX1K,64.0,100,0,NaN


### Visualize

In [11]:
from evidently.pipeline.column_mapping import ColumnMapping
from evidently.report import Report
from evidently.metrics import PrecisionTopKMetric
from evidently.metrics import RecallTopKMetric
from evidently.metrics import FBetaTopKMetric
from evidently.metrics import NDCGKMetric
from evidently.metrics import PersonalizationMetric
import warnings

warnings.filterwarnings(
    action='ignore',
    category=FutureWarning,
    module=r'evidently.metrics.recsys.precision_recall_k'
)

from src.viz import color_scheme

In [12]:
column_mapping = ColumnMapping(
    recommendations_type='rank',
    target=rating_col,
    prediction='rec_ranking',
    item_id=item_col,
    user_id=user_col
)

report = Report(metrics=[
    NDCGKMetric(k=args.top_k),
    RecallTopKMetric(k=args.top_K),
    PrecisionTopKMetric(k=args.top_k),
    FBetaTopKMetric(k=args.top_k),
    PersonalizationMetric(k=args.top_k),
], options=[color_scheme])

report.run(
    reference_data=None,
    current_data=eval_df,
    column_mapping=column_mapping
)

evidently_report_fp = f"{args.notebook_persist_dp}/evidently_report.html"
os.makedirs(args.notebook_persist_dp, exist_ok=True)
report.save_html(evidently_report_fp)

if args.log_to_mlflow:
    mlflow.log_artifact(evidently_report_fp)
    for metric_result in report.as_dict()['metrics']:
        metric = metric_result['metric']
        if metric == 'PersonalizationMetric':
            metric_value = float(metric_result['result']['current_value'])
            mlflow.log_metric(f"val_{metric}", metric_value)
            continue
        result = metric_result['result']['current'].to_dict()
        for kth, metric_value in result.items():
            mlflow.log_metric(f"val_{metric}_at_k_as_step", metric_value, step=kth)

# Clean up

In [13]:
all_params = [args]

if args.log_to_mlflow:
    for params in all_params:
        params_dict = params.dict()
        params_ = {f"{params.__repr_name__()}.{k}": v for k, v in params_dict.items()}
        mlflow.log_params(params_)

    mlflow.end_run()

2024/09/15 11:48:13 INFO mlflow.tracking._tracking_service.client: 🏃 View run 010-baseline-popular at: http://localhost:5003/#/experiments/1/runs/70eaa930c248463bb97f30b3c6140365.
2024/09/15 11:48:13 INFO mlflow.tracking._tracking_service.client: 🧪 View experiment at: http://localhost:5003/#/experiments/1.
